# Privacy Partiton Analysis

In PAT, Issuers see the number of counts associated with each request. For the sake of simplicity, consider the following scenario. Given a total number of requests N for a specific domain, and a count associated with each request, it must be true that the number of partitions of N describes all the ways that these requests could be linked to clients. For example, let N = 4. These requests could be partitioned in the following ways:

- {4}: one client asked for 4 tokens
- {1,1,1,1}: four individual clients asked for one token
- {2,2}: two clients each ask for two tokens

Depending on the size of parts, the Issuer may be able to build profiles for specific clients. In the example above, if the partition was {4}, then the Issuer knows that the same client asked for all tokens. In contrast, if the partition was {2,2}, then it knows that there are two client anonymity sets of size k=2 each. The number of distinct partitions is the number of partitions without repeating parts, where a distinct part corresponds to a unique client.  Thus, in general, the number of distinct partitions is the number of unique clients the issuer can track.

Based on this, it's worth considering the following questions:

1. How might one "round" parts to make this linkability more difficult? Is there a different strategy for conveying parts without leading to this type of linking?
2. Is it better for the Attester to learn the per-domain limits and enforce them, rather than the Issuer learn the per-client counts?

In [83]:
import math

# Partition Function P
# https://oeis.org/A000041
#
# Compute the partitions p(n) up to integer n using the recurrence
# relation defined by the generating function:
# 
#  \sum_{n=0}^{\infty} p(n) x^n = \prod_{j=1}^{\infty} (1 - x^j)^{-1}
#
# By Euler's pentagonal number theorem, this is an alternating sum
# of pentagonal number powers of its argument, yielding the recurrence
# relation
#
#   p(n) = p(n - 1) + p(n - 2) - p(n - 5) - p(n - 7) + ...
#
# with the base cases p(1) = 1 and p(2) = 2
#
# See: https://math.stackexchange.com/questions/2675382/calculating-integer-partitions
def p(n):
    partitions = [1]
    for i in range(1, n + 1):
        partitions.append(0)
        for k in range(1, i + 1):
            coeff = (-1) ** (k + 1)
            for t in [pentagonal_number(k), pentagonal_number(-k)]:
                if (i - t) >= 0:
                    partitions[i] = partitions[i] + (coeff * partitions[i - t])
    return partitions

# Compute the series 1, 2, 5, 7, 12, 15, ...
# https://oeis.org/A001318
def pentagonal_number(k):
    return int(k * (3 * k - 1) / 2)

# Partition Function Q
# https://oeis.org/A000009
#
# Compute the number of distinct partitions q(n) up to integer n using the recurrence
# relation defined by the generating function:
# 
#  \sum_{n=0}^{\infty} q(n) x^n = \prod_{j=1}^{\infty} (1 - x^{2j - 1})^{-1}
#
# This can be computed with the following recurrence relation:
#
#   q(n) = s(n) + 2*\sum_{k=1}^{\sqrt(n)} (-1)^{k+1} * q(n - k^2)
#
# with the base cases q(0) = q(1) = 1, and
#
# s(n) = (-1)^j, for n = j(3j +- 1) / 2
#      = 0, otherwise
def q(n):
    return [q_n(i) for i in range(1, n + 1)]

def q_n(n):
    if n == 0 or n == 1:
        return 1
    sqrt_n = int(math.sqrt(n)) + 1
    summation = 0
    for k in range(1, sqrt_n):
        coeff = (-1) ** (k + 1)
        summation = summation + (coeff * q_n(n - (k * k)))
    return s(n) + (2 * summation)

# s(n) = (-1)^j, for n = j(3j +- 1) / 2
#      = 0, otherwise
def s(n):
    for j in range(n):
        v1 = (j * ((3 * j) + 1)) / 2
        v2 = (j * ((3 * j) - 1)) / 2
        if n == v1 or n == v2:
            return (-1) ** j
    return 0

In [84]:
n = 16
ps_n = p(n)
qs_n = q(n)
for i in range(n):
    print("n = %d, p(n)        = %d" % (i, ps_n[i]))
    print("n = %d, q(n)        = %d" % (i, qs_n[i]))
    print("n = %d, p(n) - q(n) = %d" % (i, ps_n[i] - qs_n[i]))

n = 0, p(n)        = 1
n = 0, q(n)        = 1
n = 0, p(n) - q(n) = 0
n = 1, p(n)        = 1
n = 1, q(n)        = 1
n = 1, p(n) - q(n) = 0
n = 2, p(n)        = 2
n = 2, q(n)        = 2
n = 2, p(n) - q(n) = 0
n = 3, p(n)        = 3
n = 3, q(n)        = 2
n = 3, p(n) - q(n) = 1
n = 4, p(n)        = 5
n = 4, q(n)        = 3
n = 4, p(n) - q(n) = 2
n = 5, p(n)        = 7
n = 5, q(n)        = 4
n = 5, p(n) - q(n) = 3
n = 6, p(n)        = 11
n = 6, q(n)        = 5
n = 6, p(n) - q(n) = 6
n = 7, p(n)        = 15
n = 7, q(n)        = 6
n = 7, p(n) - q(n) = 9
n = 8, p(n)        = 22
n = 8, q(n)        = 8
n = 8, p(n) - q(n) = 14
n = 9, p(n)        = 30
n = 9, q(n)        = 10
n = 9, p(n) - q(n) = 20
n = 10, p(n)        = 42
n = 10, q(n)        = 12
n = 10, p(n) - q(n) = 30
n = 11, p(n)        = 56
n = 11, q(n)        = 15
n = 11, p(n) - q(n) = 41
n = 12, p(n)        = 77
n = 12, q(n)        = 18
n = 12, p(n) - q(n) = 59
n = 13, p(n)        = 101
n = 13, q(n)        = 22
n = 13, p(n) - q(n) = 79
n 